In [303]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-whitegrid")
import levenberg_marquardt
from matplotlib.pyplot import figure
import heston 
from importlib import reload
reload(heston)
from heston import calibrate_heston, get_tick
import datetime
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
from scipy import stats as sps
from typing import Union
import h5py
import matplotlib.dates as mdates
from tqdm.notebook import tqdm

In [2]:
%%time
raw = pd.HDFStore('../datasets/eth-20230224.h5','r').get("/eth")

CPU times: user 21.3 s, sys: 3.09 s, total: 24.3 s
Wall time: 26.2 s


In [3]:
def process_data(data):
    # only options
    df = data.copy()
    df = df[(df["instrument"].str.endswith("C")) | (df["instrument"].str.endswith("P"))].sort_values("dt")
    df["type"] = np.where(df["instrument"].str.endswith("C"), "call", "put")
    
    perpetuals = data[data["instrument"].str.endswith("PERPETUAL")][["dt", "price"]].copy()
    perpetuals = perpetuals.rename(columns = {"price": "underlying_price"}).sort_values("dt")
    
    def get_strike(x):
        return int(x.split("-")[2])
    
    def get_expiration(x):
        return x.split("-")[1]
    

    df["strike_price"] = df["instrument"].apply(get_strike)
    df["expiration"] = df["instrument"].apply(get_expiration)
    
    def unix_time_millis(dt):
        epoch = datetime.datetime.utcfromtimestamp(0)
        return int((dt - epoch).total_seconds() * 1000_000)
    
    def get_normal_date(s):
        """Function to convert date to find years to maturity"""
        monthToNum = {
            "JAN": 1,
            "FEB": 2,
            "MAR": 3,
            "APR": 4,
            "MAY": 5,
            "JUN": 6,
            "JUL": 7,
            "AUG": 8,
            "SEP": 9,
            "OCT": 10,
            "NOV": 11,
            "DEC": 12,
        }

        full_date = s.split("-")[1]
        try:
            day = int(full_date[:2])
            month = monthToNum[full_date[2:5]]
        except:
            day = int(full_date[:1])
            month = monthToNum[full_date[1:4]]
        
        year = int("20" + full_date[-2:])
        exp_date = datetime.datetime(year, month, day)
        return unix_time_millis(exp_date)
    
    df["dt"] = pd.to_datetime(df["dt"])
    perpetuals["dt"] = pd.to_datetime(perpetuals["dt"])
    
    df = pd.merge_asof(df, perpetuals, on="dt",
                       tolerance=pd.Timedelta('7 minutes'),
                       direction='nearest',)
    
    df["timestamp"] = df["dt"].apply(unix_time_millis)
    df["expiration"] = df["instrument"].apply(get_normal_date)
    df = df.rename(columns = {"price": "mark_price"})
    
    
    return df


In [4]:
%%time
start = process_data(raw)


CPU times: user 49.4 s, sys: 17.9 s, total: 1min 7s
Wall time: 1min 17s


In [325]:
df = start.copy()
# df = df[(df["timestamp"]>=1672000000000000) & (df["timestamp"]<=1672800000000000)]
df

dt  mark_price          instrument  type  \
0       2022-11-21 13:08:24.511      0.0005  ETH-25NOV22-1700-C  call   
1       2022-11-21 13:08:24.511      0.0005  ETH-25NOV22-2400-C  call   
2       2022-11-21 13:08:24.511      0.0020   ETH-25NOV22-700-P   put   
3       2022-11-21 13:08:24.511      0.0005  ETH-25NOV22-3800-C  call   
4       2022-11-21 13:08:24.511      0.0005  ETH-25NOV22-3400-C  call   
...                         ...         ...                 ...   ...   
7498621 2023-02-24 07:13:46.716      0.0300  ETH-24FEB23-1700-P   put   
7498622 2023-02-24 07:13:47.576      0.2975  ETH-24FEB23-1200-C  call   
7498623 2023-02-24 07:13:51.208      0.0015  ETH-24FEB23-1650-P   put   
7498624 2023-02-24 07:13:55.748      0.2980  ETH-24FEB23-1200-C  call   
7498625 2023-02-24 07:14:03.663      0.2975  ETH-24FEB23-1200-C  call   

         strike_price        expiration  underlying_price         timestamp  
0                1700  1669334400000000           1133.90  1669036104511000  
1                2400  1669334400000000           1133.90  1669036104511000  
2                 700  1669334400000000           1133.90  1669036104511000  
3                3800  1669334400000000           1133.90  1669036104511000  
4                3400  1669334400000000           1133.90  1669036104511000  
...               ...               ...               ...               ...  
7498621          1700  1677196800000000           1651.85  1677222826716000  
7498622          1200  1677196800000000           1651.85  1677222827576000  
7498623          1650  1677196800000000           1651.85  1677222831208000  
7498624          1200  1677196800000000           1651.85  1677222835748000  
7498625          1200  1677196800000000           1651.85  1677222843663000  

[7498626 rows x 8 columns]

In [ ]:
# df_s = pd.read_csv(f"../datasets/deribit_options_chain_2022-12-01_OPTIONS.csv")
# df_s = df_s.drop(columns = ["exchange", "open_interest", 
#                     "last_price", "bid_price", "bid_amount", "bid_iv",
#                            "ask_price", "ask_amount", "ask_iv", "underlying_index",
#                            "delta", "gamma", "vega", "theta" ,"rho"])
# currency = "ETH"
# ind = [symbol.startswith(currency) for symbol in df_s.symbol]
# df_s = df_s[ind].reset_index(drop=True)
# # df = df[df["expiration"] == 1680249600000000]
# # after expiration
# # df = df[df["timestamp"] > 1669881600000000]
# # df = df[df["timestamp"]<=1669928620213347.0]

# df = df_s.copy()
# df = df[df["expiration"].isin([1680249600000000, 1688112000000000, 
#                                1672387200000000, 1674806400000000, 1669968000000000, 
#                                1695974400000000, 1677225600000000])]

# See params dynamic

In [326]:
%%time
# calibrate base
number_of_dots = 50
timestamps_hm = []
names = ["kappa", "nu_bar", "sigma", "rho", "nu0"]

params_all = []
params_nu0 = []
params_nu0_and_nu_bar = []
params_nu0_and_k = []

errors_all = []
errors_nu0 = []
errors_nu0_and_nu_bar = []
errors_nu0_and_k = []


a = np.float64(1.0)  # kappa    a                 |  mean reversion rate | kappa (Article) | kappa (Roland's)
b = np.float64(1.3)  # v_infinity                |  long term variance | nu_bar           | alpha_bar 
c = np.float64(1.0)  # sigma                    |  volatility of volatility | sigma      | mu
rho = np.float64(-0.4)  # rho                    |  correlation between spot and vol |rho | rho
v0 = np.float64(0.65) # init variance            | initial variance | nu_0                | alpha_0
start_params = np.array([a, b, c, rho, v0])

print("Optimize first params with random start once")
calibrated_params_all, first_error_all, _ = calibrate_heston(df = df, 
                                                        start_params = start_params, 
                                                        calibration_type = "all")
calibrated_params_nu0, first_error_nu0, _ = calibrate_heston(df = df, 
                                                        start_params = start_params, 
                                                        calibration_type = "nu0")
calibrated_params_nu0_and_nu_bar, first_error_nu0_and_nu_bar, _ = calibrate_heston(df = df, 
                                                        start_params = start_params, 
                                                        calibration_type = "nu0_and_nu_bar")
calibrated_params_nu0_and_k, first_error_nu0_and_k, _ = calibrate_heston(df = df, 
                                                        start_params = start_params, 
                                                        calibration_type = "nu0_and_k")


print("All first calibrated:", [round(x, 3) for x in calibrated_params_all], "error:", first_error_all)
print("nu0 first calibrated:", [round(x, 3) for x in calibrated_params_nu0], "error:", first_error_nu0)
print("nu0_and_nu_bar calibrated:", [round(x, 3) for x in calibrated_params_nu0_and_nu_bar],
              "error:", first_error_nu0_and_nu_bar)
print("nu0_and_k calibrated:", [round(x, 3) for x in calibrated_params_nu0_and_k],
              "error:", first_error_nu0_and_k)

# Get ticks with similar distnce from available data
calibr_borders = df[(df["dt"] >= "2023-02-04") & (df["dt"] <= "2023-02-04 12:00:00")].copy()
timestamps = np.linspace(calibr_borders.timestamp.min(), calibr_borders.timestamp.max(), number_of_dots)
for timestamp in tqdm(timestamps):
    try:
        
        
        
        # zero params for each point are params from last step
        calibrated_params_all, error_all, _ = calibrate_heston(df = df, start_params = calibrated_params_all, 
                                                            timestamp = timestamp, calibration_type = "all")
        print("All calibrated:", [round(x, 3) for x in calibrated_params_all], "error:", error_all)
        # save all calib_params
        params_all.append(calibrated_params_all)
        errors_all.append(error_all)
        
        
        
        
        calibrated_params_nu0, error_nu0, _ = calibrate_heston(df = df, start_params = calibrated_params_nu0, 
                                                            timestamp = timestamp, calibration_type = "nu0")
        print("nu0 calibrated:", [round(x, 3) for x in calibrated_params_nu0], "error:", error_nu0)
        # save nu0 calib params
        params_nu0.append(calibrated_params_nu0)
        errors_nu0.append(error_nu0)
        
        
        
        
        calibrated_params_nu0_and_nu_bar, error_nu0_and_nu_bar, _ = calibrate_heston(df = df,
                                                            start_params = calibrated_params_nu0_and_nu_bar, 
                                                            timestamp = timestamp, calibration_type = "nu0_and_nu_bar")
        print("nu0nb calibrated:", [round(x, 3) for x in calibrated_params_nu0_and_nu_bar],
              "error:", error_nu0_and_nu_bar)
        # save nu0_and_nu_bar calib params
        params_nu0_and_nu_bar.append(calibrated_params_nu0_and_nu_bar)
        errors_nu0_and_nu_bar.append(error_nu0_and_nu_bar)
        
        
        
        
        calibrated_params_nu0_and_k, error_nu0_and_k, _ = calibrate_heston(df = df,
                                                        start_params = calibrated_params_nu0_and_k, 
                                                        timestamp = timestamp, calibration_type = "nu0_and_k")
        print("nu0k calibrated:", [round(x, 3) for x in calibrated_params_nu0_and_k],
              "error:", error_nu0_and_k)
        params_nu0_and_k.append(calibrated_params_nu0_and_k)
        errors_nu0_and_k.append(error_nu0_and_k)
        
        
        
        print("    ")
        # save timestame that could calibrate
        timestamps_hm.append(timestamp)
            
    except ValueError:
        # if could not calibrate, start next calibration with prev params
        print(f"Could not calibrate LM on {timestamp}")
        calibrated_params_all = start_params
        calibrated_params_nu0 = start_params

        
    except IndexError:
        print(f"Could not calibrate nu0 {timestamp}")
        calibrated_params_all = start_params
        calibrated_params_nu0 = start_params

        
    except ZeroDivisionError:
        print(f"Zero dev jacobian on {timestamp}")
        calibrated_params_all = start_params
        calibrated_params_nu0 = start_params
    

timestamps_hm = [datetime.datetime.fromtimestamp(x/1000000.0) for x in timestamps_hm]
params_all = np.array(params_all).T
params_nu0 = np.array(params_nu0).T
params_nu0_and_nu_bar = np.array(params_nu0_and_nu_bar).T
params_nu0_and_k = np.array(params_nu0_and_k).T



Optimize first params with random start once
All first calibrated: [403.548, 0.0, 59.397, -0.082, 8.148] error: 0.1827210514893162
nu0 first calibrated: [364.868, 0.574, 57.42, -0.083, 0.47] error: 0.18310369790335995
nu0_and_nu_bar calibrated: [0.0, 0.0, 2.271, -0.155, 0.47] error: 0.542459266272994
nu0_and_k calibrated: [0.236, 21.264, 6.269, -0.09, 0.47] error: 0.2145812455966803


  0%|          | 0/50 [00:00<?, ?it/s]

All calibrated: [61.239, 0.542, 12.455, -0.057, 0.479] error: 0.1179024733535795
nu0 calibrated: [45.961, 0.603, 10.858, -0.055, 0.433] error: 0.12463883398116958
nu0nb calibrated: [0.0, 0.001, 2.405, -0.084, 0.433] error: 0.2972639736951999
nu0k calibrated: [0.225, 21.96, 6.273, -0.054, 0.433] error: 0.13479510176559142
    
All calibrated: [64.957, 0.538, 13.153, -0.079, 0.488] error: 0.10477773102838209
nu0 calibrated: [46.502, 0.602, 11.165, -0.077, 0.437] error: 0.11313198273774493
nu0nb calibrated: [0.0, 0.001, 2.706, -0.123, 0.437] error: 0.2836337001287581
nu0k calibrated: [0.225, 21.434, 6.389, -0.079, 0.437] error: 0.12561641592881798
    
All calibrated: [64.344, 0.533, 13.167, -0.079, 0.487] error: 0.10419295346578773
nu0 calibrated: [46.045, 0.606, 11.193, -0.077, 0.428] error: 0.11582325042825473
nu0nb calibrated: [0.0, 0.001, 2.653, -0.122, 0.428] error: 0.30419468592794885
nu0k calibrated: [0.224, 23.121, 6.468, -0.078, 0.428] error: 0.12750653859281194
    
All calibra

All calibrated: [123.387, 0.502, 18.917, -0.079, 0.522] error: 0.11937816051756284
nu0 calibrated: [91.313, 0.545, 15.329, -0.076, 0.431] error: 0.13077581850208553
nu0nb calibrated: [0.0, 0.001, 2.472, -0.113, 0.431] error: 0.28391635210771615
nu0k calibrated: [0.221, 21.244, 6.17, -0.075, 0.431] error: 0.14997415945221673
    
All calibrated: [103.426, 0.506, 16.459, -0.087, 0.517] error: 0.11573554079911695
nu0 calibrated: [68.608, 0.557, 12.768, -0.085, 0.44] error: 0.12599922510737233
nu0nb calibrated: [0.0, 0.001, 2.771, -0.122, 0.44] error: 0.2581983254132175
nu0k calibrated: [0.221, 19.227, 6.072, -0.085, 0.44] error: 0.141406786077919
    
All calibrated: [76.505, 0.52, 14.522, -0.067, 0.501] error: 0.12094568627808258
nu0 calibrated: [51.517, 0.592, 11.803, -0.064, 0.43] error: 0.13287510751604914
nu0nb calibrated: [0.0, 0.001, 2.714, -0.079, 0.43] error: 0.296915433983687
nu0k calibrated: [0.221, 23.018, 6.553, -0.06, 0.43] error: 0.14400898013229554
    
All calibrated: [89

In [327]:
fig, axs =  plt.subplots(nrows=3, ncols=2, figsize=(20, 15))

axs[0, 0].plot(timestamps_hm, params_all[0])
axs[0, 0].plot(timestamps_hm, params_nu0[0])
axs[0, 0].plot(timestamps_hm, params_nu0_and_nu_bar[0])
axs[0, 0].plot(timestamps_hm, params_nu0_and_k[0])
axs[0, 0].set_title(r"$\kappa$ - mean reversion rate")

axs[0, 1].plot(timestamps_hm, params_all[1])
axs[0, 1].plot(timestamps_hm, params_nu0[1])
axs[0, 1].plot(timestamps_hm, params_nu0_and_nu_bar[1])
axs[0, 1].plot(timestamps_hm, params_nu0_and_k[1])
axs[0, 1].set_title(r"$\bar{\nu}$ - long term variance")

axs[1, 0].plot(timestamps_hm, params_all[2], label = "all params")
axs[1, 0].plot(timestamps_hm, params_nu0[2], label = r"$\nu_0$")
axs[1, 0].plot(timestamps_hm, params_nu0_and_nu_bar[2], label = r"$\nu_0, \bar{\nu}$")
axs[1, 0].plot(timestamps_hm, params_nu0_and_k[2], label = r"$\nu_0, \kappa$")
axs[1, 0].set_title(r"$\sigma$ - vol of vol")

axs[1, 1].plot(timestamps_hm, params_all[3])
axs[1, 1].plot(timestamps_hm, params_nu0[3])
axs[1, 1].plot(timestamps_hm, params_nu0_and_nu_bar[3])
axs[1, 1].plot(timestamps_hm, params_nu0_and_k[3])
axs[1, 1].set_title("ρ -rho")

axs[2, 0].plot(timestamps_hm, params_all[4])
axs[2, 0].plot(timestamps_hm, params_nu0[4])
axs[2, 0].plot(timestamps_hm, params_nu0_and_nu_bar[4])
axs[2, 0].plot(timestamps_hm, params_nu0_and_k[4])
axs[2, 0].set_title(r"$\nu_0$ - initial variance")

axs[2, 1].plot(timestamps_hm, errors_all)
axs[2, 1].plot(timestamps_hm, errors_nu0)
axs[2, 1].plot(timestamps_hm, errors_nu0_and_nu_bar)
axs[2, 1].plot(timestamps_hm, errors_nu0_and_k)
axs[2, 1].set_title("error")



# for i in range(3):
#     for j in range(2):
#         for exp in expirarions:
#             axs[i, j].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
#             axs[i, j].axvline(x=exp, color = "black", linestyle  = "--")
#             if i == j == 0:
#                 axs[i, j].axvline(x=exp, color = "black", linestyle  = "--", label = exp)
#                 pass


plt.figlegend() 
fig.suptitle("Heston params dynamic")
plt.gcf().autofmt_xdate()

plt.show()

# Vol smile

In [272]:
stamp = df.sample(1).iloc[0].timestamp
# print(df.timestamp.max() - df.timestamp.min())
print(stamp)

1675118523859000


In [306]:
# 1673321843011000

needed_stamp = 1677222826716000
a = np.float64(1.0)  # kappa    a                 |  mean reversion rate | kappa (Article) | kappa (Roland's)
b = np.float64(1.3)  # v_infinity                |  long term variance | nu_bar           | alpha_bar 
c = np.float64(1.0)  # sigma                    |  volatility of volatility | sigma      | mu
rho = np.float64(-0.4)  # rho                    |  correlation between spot and vol |rho | rho
v0 = np.float64(0.65) # init variance            | initial variance | nu_0                | alpha_0
start_params = np.array([a, b, c, rho, v0])

pars_all, err_all, C_all = calibrate_heston(df = df, start_params = start_params, timestamp = needed_stamp,
                                calibration_type= "all")

pars_nu0, err_nu0, C_nu0 = calibrate_heston(df= df, start_params = start_params, timestamp = needed_stamp,
                                calibration_type= "nu0")

pars_and_nu_bar, err_and_nu_bar, C_and_nu_bar = calibrate_heston(df= df, start_params = start_params, 
                                                                 timestamp = needed_stamp,
                                calibration_type= "nu0_and_nu_bar")

pars_nu0_and_k, err_nu0_and_k, C_nu0_and_k = calibrate_heston(df= df, start_params = start_params, 
                                                                 timestamp = needed_stamp,
                                calibration_type= "nu0_and_k")


print("All:")
print([round(x, 3) for x in pars_all])
print(err_all)
print("   ")
print("nu0:")
print([round(x, 3) for x in pars_nu0])
print(err_nu0)
print("   ")
print("nu0_and_nu_bar:")
print([round(x, 7) for x in pars_and_nu_bar])
print(err_and_nu_bar)
print("   ")
print("nu0_and_k:")
print([round(x, 3) for x in pars_nu0_and_k])
print(err_nu0_and_k)
print("   ")

All:
[403.548, 0.0, 59.397, -0.082, 8.148]
0.1827210514893162
   
nu0:
[364.868, 0.574, 57.42, -0.083, 0.47]
0.18310369790335995
   
nu0_and_nu_bar:
[0.0001, 0.0003065, 2.270671, -0.1546851, 0.4698374]
0.5424592662729518
   
nu0_and_k:
[0.236, 21.264, 6.269, -0.09, 0.47]
0.21458124441428716
   


In [324]:
figure(figsize=(15, 7), dpi=80)

exp = C_all.sample(1).expiration.iloc[0]
C_sing_exp_all = C_all[C_all["expiration"] == exp]
C_sing_exp_nu0 = C_nu0[C_nu0["expiration"] == exp]
C_sing_exp_and_nu_bar = C_and_nu_bar[C_and_nu_bar["expiration"] == exp]
C_sing_exp_nu0_and_k = C_nu0_and_k[C_nu0_and_k["expiration"] == exp]


plt.scatter(C_sing_exp_all["strike_price"], C_sing_exp_all["market_iv"], label = "market", linewidths = 5)
plt.scatter(C_sing_exp_all["strike_price"], C_sing_exp_all["calibrated_iv"], label = "all params")
plt.scatter(C_sing_exp_nu0["strike_price"], C_sing_exp_nu0["calibrated_iv"], label = r"$\nu_0$")
plt.scatter(C_sing_exp_and_nu_bar["strike_price"], C_sing_exp_and_nu_bar["calibrated_iv"], label = r"$\nu_0, \bar{\nu}$")
plt.scatter(C_sing_exp_nu0_and_k["strike_price"], C_sing_exp_nu0_and_k["calibrated_iv"], label = r"$\nu_0, \kappa$")
plt.vlines([C_sing_exp_all.underlying_price.mean()], C_sing_exp_all.market_iv.min()-5, 
           C_sing_exp_all.market_iv.max() + 15, color = "black", linestyles = "dotted")

plt.xlabel("strike, USD")
plt.ylabel("iv, %")
plt.legend()
plt.show()

In [ ]:
# single_instr = df[df["instrument"] == df.sample(1).instrument.iloc[0]]
# plt.plot(single_instr["dt"], single_instr["mark_price"])
# plt.plot(df["dt"], df["underlying_price"])